In [113]:
import pandas as pd
import requests
import time
import math
import json
from bs4 import BeautifulSoup

df = pd.read_excel('./test.xlsx')

In [114]:
# bs4提取文本
def get_text_bs(html):
    tree = BeautifulSoup(html, 'lxml')

    body = tree.body
    if body is None:
        return None

    for tag in body.select('script'):
        tag.decompose()
    for tag in body.select('style'):
        tag.decompose()

    text = body.get_text(separator='')
    return text

# 定义获取详细回答的通用函数
def getAnswer(url, count):
    # 必要参数
    num = url[-9:]
    freq = math.floor(count/5)
    outArr = []
    # 网络请求参数
    baseUrl = 'https://www.zhihu.com/api/v4/questions/{}/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cattachment%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Cis_labeled%2Cpaid_info%2Cpaid_info_content%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_recognized%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cvip_info%2Cbadge%5B%2A%5D.topics%3Bdata%5B%2A%5D.settings.table_of_content.enabled&limit=5&offset={}&platform=desktop&sort_by=default'
    headers = {
        'Host':'www.zhihu.com',
        'Connection':'keep-alive',
        'x-zse-93':'101_3_2.0',
        'x-ab-param':'se_ffzx_jushen1=0;pf_adjust=1;zr_slotpaidexp=1;tp_topic_style=0;tp_zrec=1;tp_contents=2;pf_noti_entry_num=2;zr_expslotpaid=1;top_test_4_liguangyi=1;qap_question_visitor= 0;tp_dingyue_video=0;qap_question_author=0',
        'x-ab-pb':'Cr4B4wRWBYwCwQSeBaED9AMzBMoFRwD2AvQLGwDpBHUENwVQA4kM1wIyBaIDtQvKAhEF4AvHAg4FsQVpAcwCGQUBC+wKOwK3A08DjgNXBM8L2gRDBXQBCwRFBFEF1wsqA+EEmws/AA8LNAwpBYAFUgtXA6ADVQUyA6YE0QQ3DPgDNATgBMIFVgy0CmoBCgQ/BdwL5ApyAxIFUgXzA0IEtADWBLIFFQUzBQcMYAt9AmQEfwVDALkCFAVAAYQCnwLYAhJfAgAAAAAAAAAAAAAAAAEAAAAAAAAAAwAAAAAAAAABAAEBAAAAAAELAQAAAAEAAAEEAgABAAAAAQAAAAAABQEBAAEAAQAAAQAAAAEAAAAAAAAAAgEBAAEBARUVAgEAAgA=',
        'sec-ch-ua-mobile':'?0',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
        'x-zst-81':'3_2.0aR_sn77yn6O92wOB8hPZnQr0EMYxc4f18wNBUgpTr7tuSRFqK6P0E69y-LS9-hp1DufI-we8gGHPgJO1xuPZ0GxCTJHR7820XM20cLRGDJXfgGCBxupMuD_Ie8FL7AtqM6O1VDQyQ6nxrRPCHukMoCXBEgOsiRP0XL2ZUBXmDDV9qhnyTXFMnXcTF_ntRueThXY9669GtqpBpG2Y4Ue1ywX1nqwVh9g8W9xBXcH1SBV_ZBYm1q3_wqOy2BYPvASLvrNfuDUmJ93_Yw3O1MxqADgmph2BkvOO0JSBVbLMQuosIq3_5Co1jwNMSTF_kCH9iCLL9wLfFvH8rRFB8DO8pup0Ecx8-9NmTvcVXDuGQJx1VuY05qfzeH3C3hc1QMSCYMOCrDC02qpm-bO88RCGoq3BawCmFqpMQBg9eTHCc9tLICV187SMQQV_twCYACLfsGL0-remJqLm1DpO8HLGHhV8jhp_r_V0uqXC2J9C',
        'x-requested-with':'fetch',
        'x-zse-96':'2.0_aLFBre9ygutxS0YB1LxB2irqFBxxFwxqGRtybHXBgBFx',
        'sec-ch-ua':'"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
        'sec-ch-ua-platform':'"Windows"',
        'Accept':'*/*',
        'Sec-Fetch-Site':'same-origin',
        'Sec-Fetch-Mode':'cors',
        'Sec-Fetch-Dest':'empty',
        'Referer':'https://www.zhihu.com/question/20970974',
        'Accept-Encoding':'gzip, deflate, br',
        'Accept-Language':'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie':'_zap=7c6b1178-7ecf-4461-8102-605774e55301; d_c0="AICSY5B5gxGPTrOMPR5UTZEJEa6iT3yPENQ=|1593660904"; _ga=GA1.2.1934862013.1593660905; _xsrf=ffe04MllUDtjKNWzNio25uYOXrASgbsq; __snaker__id=tAeJ0FlSoB76vsVm; _9755xjdesxxd_=32; YD00517437729195%3AWM_TID=nMOwYnz4Wx1BUQVRVEJ%2F1MBvS%2BaVFbM8; __utmv=51854390.100--|2=registration_date=20210317=1^3=entry_date=20200709=1; __utmz=51854390.1626961624.2.2.utmcsr=zhihu.com|utmccn=(referral)|utmcmd=referral|utmcct=/question/473802687; gdxidpyhxdE=yI8wbeZ%5CHpQGXIQ%2BguZ%5CCRgqa5dRop%5CLS3HsQNmUpeZk9hd8YxAHTS3bzmKSXMwx3PB9tXZJ%2FViYLs1mvumf%2B1yi9pmNqNvjwadnXItJucSaxl9aldBDRcC3AoeN4wB%2FbN8neBDibevqdIi1RZYlEoKZb2bG6KqHhXPTUneGu50bGsqL%3A1634660927156; YD00517437729195%3AWM_NI=c90cP4Q2oGizDeBphXp%2BeXehv0y%2B2S9SNiiZqrmBvmncbVdnkNYQr9D%2BaNiNZFLOp0lCbA2WpIVdwiSMjRxRrxlJOvIrK5ZKhuHHcZRQvVBgK2gxbGVPdaQEuV5FaLgaNE4%3D; YD00517437729195%3AWM_NIKE=9ca17ae2e6ffcda170e2e6eea2c460a6ae8fa4e65a8ae78ab3c14e838f8e84f4799bba9daeec5083f18f85d12af0fea7c3b92aa1e70084d34093eea1a8ec7288bb8e91eb54b4f5abb9e864b6f0b7ccb159fb8d828aec52b3aea3a3f567aebdaeb7e766a99297b2f13388f5afafec469a9a84b0f33cb09daea2b15db4979a85f6639193a6b3fb5ab2a8a4abce7e8bae85d6d941acab88a9e252928fbab7c16d89eea999e14d9690ac82d44fb78afba7e448a5bf82a6b737e2a3; l_cap_id="N2M5ZjIxOGI0MGVlNGRlMzgyMDUzM2RiMzcwYjkxN2U=|1634660049|987bf4c70953a7ef070519d816b97bf2d232a512"; r_cap_id="ODdlMTgwMDI2ZjQ5NGUyOWE2MjgxYzAyZWJjZTc1NjQ=|1634660049|017dca5083291d30771993d1b47a7eb8fc0e12ae"; cap_id="NGU4MmZkNTFlY2NlNDU0ZTgzNjQyNzUyZjFmMzg4Y2Q=|1634660049|521fb82a5206009b77722e9429ccc333a09e38f5"; captcha_session_v2="2|1:0|10:1634660114|18:captcha_session_v2|88:SUp2bTZZTjM3dHFDRERJQmFBWC9aemJ6WXhkSjBCL05VUG5WQlpld09wdk0zTzRSdG1CczlhWTloVDRTQUV1Mw==|3533890e2d2c98b2603d05d00ca5ac6ee12b7bd0ea0cfc5795beb03b837d1270"; captcha_ticket_v2="2|1:0|10:1634660122|17:captcha_ticket_v2|704:eyJ2YWxpZGF0ZSI6IkNOMzFfMHZSLjV4cFhKZ2RhMkNJY0JoMDhma2pLZEs3YXljTDgyb1BtSy11SF9aSmFhN3lpT1R6WGlMbUVqQ1RWQUNjX2JHdzdMbmg4eDB0YWRKRWk2LWJTWnlmZ1pXVXRsOGJmR20uZ3ROVkh3U0lYWnZBMW80cGZDdE5LUmdSdnF5RjJzZmljb1BkbVJ4aVN6WHRuc256MEVCbUUwUVdqZ3d5cGVXeFkuLU1aLlJCQzFIRGtzaTBVcmp1aVk0cS5LbWNDaEQ1QTB5R2ZialhlZmxwRXpMaFBXd25tZUU2NGh2Q24wMDJsMmFfd0ZfTElJb3ZkN0FJTjBidkJaUVJyZlBnb0VOVlRrLU92LVRxNFRCc3RqeVZNYUFqcGEudnNTcndlcmpjSFk0al9MS19NRDFtMVJDXzFXWFphNERsc1lZVVNiRnlqdW1KdFFIcVlsUkxIXy1ra1Y3cTJVRGhKSEZxMGx1a1lNcHZldVcuMHRJc2xqNXNwRC1PNXVvVnd5cERieWVXaHpnUGJ0NHNaNkNKaXJJZC0tcGcwZHc5RVdKcDVUeWs5Zng5QXlINjUxR21DNWtuWi1rRDhCUlUtMEV6c2JSTW9JY1N3SjcySEZoLkVmbjI3RnRnQnAuVXFWaGJ5eHJjY2JyUHhYLm9NS3FULkZWX3l6RC1GeTBaMyJ9|6f750776e2778fb05170e69ba09def54c9f66701cd6c7aa7e105b164e67a8c0d"; z_c0="2|1:0|10:1634660139|4:z_c0|92:Mi4xZWlTakt3QUFBQUFBZ0pKamtIbURFU1lBQUFCZ0FsVk5LejFjWWdBNkJ3Q0piUE4xTks4Q2Z4MHg2bEoyY0s4a0NB|f65065f1a250e2cfc49e57d444ac431c50107835a3ca3cc46307691af77e2810"; q_c1=ad266df5c57c402098b8dbc61971e450|1634660266000|1594278698000; __utma=51854390.1934862013.1593660905.1634660267.1634699792.8; tst=r; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1634563375,1634660025,1634699788,1634716706; SESSIONID=u11jjwjqFSWRANBpTcqcKkm5Mip3O82Aq4uM5RPnBb7; JOID=VV0SAU_eWfGpKb7uPtt4qDflZX4vqWrFxUDLhkqbCKT-bvOuTPiSnc0ovuk9-f3A6mvV6nc2K3_CqOMP5-9TLUY=; osd=W10UCknQWfeiL7DuONB-pjfjbnghqWzOw07LgEGdBqT4ZfWgTP6Zm8MouOI79_3G4W3b6nE9LXHCrugJ6e9VJkA=; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1634720111; KLBRSID=37f2e85292ebb2c2ef70f1d8e39c2b34|1634720152|1634716215',
        'Content-Type':'text/plain'
    }
    # 按频次遍历并提取信息
    for i in range(1, freq+1):
#         try:
            reqUrl = baseUrl.format(num, i*5)
            res = requests.get(reqUrl, headers=headers)
            loadedData = json.loads(res.text)['data']
            for i in loadedData:
                author = i['author']['name']
                authorUrl = i['author']['url']
                isOrg = i['author']['is_org']
                followerCount = i['author']['follower_count']
                gender = i['author']['gender']
                commentNum = i['comment_count']
                voteupNum = i['voteup_count']
                answerUrl = i['url']
                # 解析时间
                timeStamp = i['updated_time']
                timeArray = time.localtime(int(timeStamp)) 
                updatedTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
                # 文本提取(调用了一个第三方库)
                text = i['content']
                content = get_text_bs(text)
                # 问题信息
                questionTitle = i['question']['title']
                questionUrl = i['question']['url']
                questionID = i['question']['id']
                quetionCreatedTimestamp = i['question']['created']
                questionTimeArray = time.localtime(int(quetionCreatedTimestamp)) 
                quetionCreatedTime = time.strftime("%Y-%m-%d %H:%M:%S", questionTimeArray)

                outArr.append([author, authorUrl, isOrg, followerCount, gender, commentNum, voteupNum, answerUrl, updatedTime, content, questionTitle, questionUrl, questionID, quetionCreatedTime])
#         except:
#             outArr.append([None, None, None, None, None, None, None, None, None, None, None, None, None, None])
        
    return outArr

In [115]:
out = pd.DataFrame(columns=('作者', '作者主页', '是否为机构号', '关注者总数', '性别', '评论数', '赞数', '回答链接', '回答时间', '回答内容', '问题标题', '问题链接', '问题ID', '问题提问时间'))
count = 0
for i in df['问题主页']:
    resArr = getAnswer(i, 10)
    for v in resArr:
        print(count)
        out.loc[count] = v
        count += 1
print('Scrape Over')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
out.to_excel('./comments.xlsx')